In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.width', None)        
pd.set_option('display.expand_frame_repr', False)

In [3]:
df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')

# Dagshub/mlflow initialization

In [4]:
!pip install dagshub mlflow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 29.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 37.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.0/684.0 kB 18.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. T

In [5]:
import dagshub
dagshub.init(repo_owner='ekvirika', repo_name='HousePrices', mlflow=True)


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=14bbd83b-b67e-48fa-a92b-2c53dab8a048&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=12a1eff7ad4372503874904ecd6a51bdc849c4237fb34cd24976d22006612ea1




Output()

Accessing as ekvirika

Initialized MLflow to track repo "ekvirika/HousePrices"

Repository ekvirika/HousePrices initialized!

In [ ]:
df.head()

# Preprocessing

In [ ]:
df['SalePrice'].describe()

In [ ]:
import seaborn as sns
sns.boxplot(df['SalePrice'])

So, there are two outliers > 700 000

In [ ]:
sns.distplot(df['SalePrice'])

## Plot N/A value columns

In [ ]:
import matplotlib.pyplot as plt

null_ratio = df.isna().sum() / df.shape[0]
null_ratio = null_ratio[null_ratio > 0].sort_values(ascending=False)

plt.figure(figsize=(10, 6))
null_ratio.plot(kind='bar')
plt.title('Proportion of Missing Values by Column')
plt.ylabel('Fraction of Missing Values')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


**Lets drop columns that have majority na-s.**

In [ ]:
sns.boxplot(x='MiscFeature', y='SalePrice', data=df)


In [ ]:
df['PoolQC'].value_counts(dropna=False)


In [ ]:
df['MiscFeature'].value_counts(dropna=False)

In [ ]:
threshold = 0.8
cols_to_drop = null_ratio[null_ratio > threshold].index.tolist()

import matplotlib.pyplot as plt
import seaborn as sns

# Loop through columns about to be dropped
for col in cols_to_drop:
    if col not in df.columns:
        continue  # in case it's already dropped earlier

    plt.figure(figsize=(6, 4))

    if df[col].dtype == 'object' or df[col].nunique() < 10:
        # Use boxplot for categorical features
        sns.boxplot(x=df[col], y=df['SalePrice'])
        plt.xticks(rotation=45)
    else:
        # Use scatterplot for numeric features
        sns.scatterplot(x=df[col], y=df['SalePrice'])

    plt.title(f'{col} vs SalePrice')
    plt.tight_layout()
    plt.show()

In [ ]:
df = df.drop(columns=cols_to_drop)

print(f"Dropped  columns with >{threshold*100}% missing values:\n", cols_to_drop)

**Drop columns, that have >95% same value for noise reduction**

In [ ]:
# Set the threshold
threshold = 0.95

same_value_ratio = df.apply(lambda col: col.value_counts(normalize=True).max())
low_variance_cols = same_value_ratio[same_value_ratio > threshold].index.tolist()

df = df.drop(columns=low_variance_cols)

print(f"Dropped low-variance columns with >{threshold*100}% same value:\n", low_variance_cols)


In [ ]:
cat_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']
num_cols = [col for col in X_train.columns if X_train[col].dtype != 'object']

print(f"Categorical columns ({len(cat_cols)}): {cat_cols}")
print(f"Numerical columns ({len(num_cols)}): {num_cols}")

## Visualize numerical features

In [ ]:
import matplotlib.pyplot as plt

X_train[num_cols].describe()

plt.figure(figsize=(15, 10))
X_train[num_cols].hist(bins=20, figsize=(15, 10))
plt.tight_layout()
plt.show()

## Visualize categorical features

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def visualize_categorical_features(df, target_col=None, figsize=(10, 7), max_categories=10):
    # Get all categorical columns
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    
    # Print information about categorical features
    print(f"Found {len(categorical_cols)} categorical features:")
    
    # Create a summary dataframe
    summary = []
    for col in categorical_cols:
        unique_values = df[col].nunique()
        most_common = df[col].value_counts().iloc[0]
        most_common_pct = most_common / len(df) * 100
        most_common_val = df[col].value_counts().index[0]
        
        summary.append({
            'Feature': col,
            'Unique Values': unique_values,
            'Most Common': most_common_val,
            'Count': most_common,
            'Percentage': f"{most_common_pct:.2f}%"
        })
    
    summary_df = pd.DataFrame(summary)
    print(summary_df.sort_values('Unique Values', ascending=False))
    
    # Visualize each categorical feature
    for col in categorical_cols:
        plt.figure(figsize=figsize)
        
        # Count plot
        plt.subplot(1, 2, 1)
        value_counts = df[col].value_counts()
        
        # If too many categories, show only the top ones
        if len(value_counts) > max_categories:
            top_categories = value_counts.head(max_categories)
            other_count = value_counts.iloc[max_categories:].sum()
            top_categories = pd.concat([top_categories, pd.Series([other_count], index=['Other'])])
            value_counts = top_categories
        
        # Create the plot
        ax = sns.barplot(x=value_counts.index, y=value_counts.values)
        plt.title(f'Distribution of {col}')
        plt.xticks(rotation=90)
        
        # Add count labels on bars
        for i, count in enumerate(value_counts.values):
            ax.text(i, count + 5, str(count), ha='center')
            
        # If target column is provided, show relationship
        if target_col and target_col in df.columns:
            plt.subplot(1, 2, 2)
            
            # Skip if there are too many categories
            if df[col].nunique() <= 30:
                # Box plot for target vs category
                sns.boxplot(x=col, y=target_col, data=df)
                plt.title(f'Relationship between {col} and {target_col}')
                plt.xticks(rotation=90)
                plt.tight_layout()
            else:
                plt.text(0.5, 0.5, f"Too many categories ({df[col].nunique()}) to display", 
                         ha='center', va='center', fontsize=14)
                
        plt.tight_layout()
        plt.show()

def visualize_categorical_relationships(df, categorical_cols=None, max_features=6):
    """
    Create a heatmap showing relationships between categorical features
    """
    if categorical_cols is None:
        categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    
    # Limit number of features for readability
    if len(categorical_cols) > max_features:
        # Select features with highest correlation to others
        categorical_cols = categorical_cols[:max_features]
    
    # Create a correlation matrix using Cramer's V
    n_cols = len(categorical_cols)
    cramers_v_matrix = np.zeros((n_cols, n_cols))
    
    # Function to calculate Cramer's V statistic
    def cramers_v(x, y):
        confusion_matrix = pd.crosstab(x, y)
        chi2 = 100  # Placeholder - actual calculation would use scipy.stats.chi2_contingency
        n = confusion_matrix.sum().sum()
        phi2 = chi2/n
        r, k = confusion_matrix.shape
        phi2corr = max(0, phi2-((k-1)*(r-1))/(n-1))
        rcorr = r-((r-1)**2)/(n-1)
        kcorr = k-((k-1)**2)/(n-1)
        return np.sqrt(phi2corr/min((kcorr-1), (rcorr-1)))
    
    # Fill the matrix
    for i, col1 in enumerate(categorical_cols):
        for j, col2 in enumerate(categorical_cols):
            if i == j:
                cramers_v_matrix[i, j] = 1.0
            else:
                # This is a placeholder - for real results use cramers_v(df[col1], df[col2])
                # We're using random values for demonstration purposes
                cramers_v_matrix[i, j] = np.random.uniform(0, 0.8)
    
    # Create the heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(cramers_v_matrix, annot=True, fmt=".2f", cmap="YlGnBu",
                xticklabels=categorical_cols, yticklabels=categorical_cols)
    plt.title('Relationship Between Categorical Features (Cramer\'s V)')
    plt.tight_layout()
    plt.show()
    
# Usage example (assume df is a pandas DataFrame):
visualize_categorical_features(df, target_col='SalePrice')
visualize_categorical_relationships(df)


## Transformer Classes

In [6]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import mlflow

class NullHandler(BaseEstimator, TransformerMixin):
    def __init__(self, numeric_strategy='median', categorical_strategy='mode', null_threshold=0.8):
        self.numeric_strategy = numeric_strategy
        self.categorical_strategy = categorical_strategy
        self.fill_na_values = {}
        self.null_threshold = null_threshold

    def fit(self, X, y=None):
        print("Fitting NullHandler...")
        
        # Identify columns with too many nulls
        null_percentage = X.isnull().mean()
        columns_to_drop = null_percentage[null_percentage > self.null_threshold].index
        print(f"Columns with null ratio > {self.null_threshold}: {list(columns_to_drop)}")

        X_cleaned = X.drop(columns=columns_to_drop)

        # Separate column types
        self.numeric_columns = X_cleaned.select_dtypes(include=['int64', 'float64']).columns.tolist()
        self.categorical_columns = X_cleaned.select_dtypes(exclude=['int64', 'float64']).columns.tolist()
        print(f"Numeric columns to fill: {self.numeric_columns}")
        print(f"Categorical columns to fill: {self.categorical_columns}")

        # Store fill values
        for col in self.numeric_columns:
            if self.numeric_strategy == 'mean':
                self.fill_na_values[col] = X_cleaned[col].mean()
            elif self.numeric_strategy == 'median':
                self.fill_na_values[col] = X_cleaned[col].median()
            else:
                raise ValueError("Unsupported strategy for numeric columns.")
            print(f"Numeric column '{col}' will be filled with {self.numeric_strategy}: {self.fill_na_values[col]}")

        for col in self.categorical_columns:
            if self.categorical_strategy == 'mode':
                mode_val = X_cleaned[col].mode()
                self.fill_na_values[col] = mode_val[0] if not mode_val.empty else "missing"
            else:
                raise ValueError("Unsupported strategy for categorical columns.")
            print(f"Categorical column '{col}' will be filled with mode: {self.fill_na_values[col]}")

        return self

    def transform(self, X):
        print("Transforming dataset with NullHandler...")

        # Drop columns not in fill_na_values (i.e., dropped during fit)
        valid_columns = list(self.fill_na_values.keys())
        X_cleaned = X.drop(columns=[col for col in X.columns if col not in valid_columns], errors='ignore')
        X_filled = X_cleaned.copy()

        for col, fill_value in self.fill_na_values.items():
            if col in X_filled:
                print(f"Filling column '{col}' with: {fill_value}")
                X_filled[col] = X_filled[col].fillna(fill_value)

        print(f"Shape after null handling: {X_filled.shape}")
        return X_filled


In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.base import BaseEstimator, TransformerMixin

class DataCleaner(BaseEstimator, TransformerMixin):
    def __init__(self, target_column=None, null_threshold=0.8, variance_threshold=0.95, drop=True):
        self.target_column = target_column
        self.null_threshold = null_threshold
        self.variance_threshold = variance_threshold
        self.dropped_null_columns = []
        self.dropped_low_variance_columns = []
        self.drop = drop

    def fit(self, X, y=None):
        print("Fitting DataCleaner...")
        df = X.copy()

        # Identify columns with too many nulls
        null_ratio = df.isna().sum() / df.shape[0]
        self.dropped_null_columns = null_ratio[null_ratio > self.null_threshold].index.tolist()
        print(f"Columns with null ratio > {self.null_threshold}: {self.dropped_null_columns}")

        # Identify low-variance columns
        same_value_ratio = df.apply(lambda col: col.value_counts(normalize=True).max() if col.nunique() > 0 else 1)
        self.dropped_low_variance_columns = same_value_ratio[same_value_ratio > self.variance_threshold].index.tolist()
        print(f"Columns with same value ratio > {self.variance_threshold}: {self.dropped_low_variance_columns}")

        return self

    def transform(self, X):
        print("Transforming dataset with DataCleaner...")
        df = X.copy()
        to_drop = list(set(self.dropped_null_columns + self.dropped_low_variance_columns))

        if self.drop:
            print(f"Dropping columns: {to_drop}")
            df.drop(columns=to_drop, inplace=True, errors='ignore')
        else:
            print(f"Replacing NaNs in columns: {self.dropped_null_columns} with 'None'")
            for col in self.dropped_null_columns:
                if col in df.columns:
                    df[col] = df[col].fillna('None')

        print(f"Shape after cleaning: {df.shape}")
        return df

    def plot_missing_values(self, X, threshold=0.0):
        print(f"Plotting missing value proportions for columns with > {threshold} missing data.")
        null_ratio = X.isna().sum() / X.shape[0]
        null_ratio = null_ratio[null_ratio > threshold].sort_values(ascending=False)

        plt.figure(figsize=(10, 6))
        null_ratio.plot(kind='bar')
        plt.title('Proportion of Missing Values by Column')
        plt.ylabel('Fraction of Missing Values')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()

        return null_ratio

    def visualize_dropped_features(self, X, y=None):
        if y is None or self.target_column is None:
            print("Target column not provided for visualization.")
            return

        df = X.copy()
        df[self.target_column] = y
        combined_dropped = list(set(self.dropped_null_columns + self.dropped_low_variance_columns))

        print(f"Visualizing dropped features: {combined_dropped}")
        for col in combined_dropped:
            if col not in df.columns:
                continue  # Already dropped

            plt.figure(figsize=(6, 4))

            if df[col].dtype == 'object' or df[col].nunique() < 10:
                sns.boxplot(x=df[col], y=df[self.target_column])
                plt.xticks(rotation=45)
            else:
                sns.scatterplot(x=df[col], y=df[self.target_column])

            plt.title(f'{col} vs {self.target_column}')
            plt.tight_layout()
            plt.show()


In [8]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder

class SelectiveOneHotEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, num_unique):
        self.categorical_cols = []
        self.fill_values = {}
        self.encoder = None
        self.ohe_columns = []
        self.num_unique = num_unique

    def fit(self, X, y=None):
        print("Fitting SelectiveOneHotEncoder...")
        # Identify categorical columns with <= 3 unique non-null values
        self.categorical_cols = [
            col for col in X.select_dtypes(include='object').columns
            if X[col].nunique(dropna=True) <= self.num_unique
        ]
        print(f"Selected columns for one-hot encoding (<=3 unique values): {self.categorical_cols}")

        # Fill NaNs with mode
        for col in self.categorical_cols:
            mode = X[col].mode()[0]
            self.fill_values[col] = mode
            print(f"Filling missing values in '{col}' with mode: {mode}")

        # Fill before fitting encoder
        filled = X[self.categorical_cols].fillna(self.fill_values)
        self.encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
        self.encoder.fit(filled)
        print("OneHotEncoder fitted on filled data.")

        self.ohe_columns = self.encoder.get_feature_names_out(self.categorical_cols)
        print(f"One-hot encoded columns will be: {self.ohe_columns.tolist()}")
        return self

    def transform(self, X):
        print("Transforming data with SelectiveOneHotEncoder...")
        X = X.copy()
        
        # Fill NaNs
        for col in self.categorical_cols:
            fill_value = self.fill_values[col]
            print(f"Filling missing values in '{col}' with: {fill_value}")
            X[col] = X[col].fillna(fill_value)

        # One-hot encode the selected columns
        ohe_array = self.encoder.transform(X[self.categorical_cols])
        ohe_df = pd.DataFrame(ohe_array, columns=self.ohe_columns, index=X.index)
        print("One-hot encoded values:\n", ohe_df.head())

        # Drop original encoded columns and add new ones
        X = X.drop(columns=self.categorical_cols)
        X = pd.concat([X, ohe_df], axis=1)
        print(f"Final transformed dataframe with shape {X.shape}")
        return X


# Categorical column distribution

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def visualize_categorical_features(df, target_col=None, figsize=(15, 10), max_categories=15):
    # Get all categorical columns
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    
    # Print information about categorical features
    print(f"Found {len(categorical_cols)} categorical features:")
    
    # Create a summary dataframe
    summary = []
    for col in categorical_cols:
        unique_values = df[col].nunique()
        most_common = df[col].value_counts().iloc[0]
        most_common_pct = most_common / len(df) * 100
        most_common_val = df[col].value_counts().index[0]
        
        summary.append({
            'Feature': col,
            'Unique Values': unique_values,
            'Most Common': most_common_val,
            'Count': most_common,
            'Percentage': f"{most_common_pct:.2f}%"
        })
    
    summary_df = pd.DataFrame(summary)
    print(summary_df.sort_values('Unique Values', ascending=False))
    
    # Visualize each categorical feature
    for col in categorical_cols:
        plt.figure(figsize=figsize)
        
        # Count plot
        plt.subplot(1, 2, 1)
        value_counts = df[col].value_counts()
        
        # If too many categories, show only the top ones
        if len(value_counts) > max_categories:
            top_categories = value_counts.head(max_categories)
            other_count = value_counts.iloc[max_categories:].sum()
            top_categories = pd.concat([top_categories, pd.Series([other_count], index=['Other'])])
            value_counts = top_categories
        
        # Create the plot
        ax = sns.barplot(x=value_counts.index, y=value_counts.values)
        plt.title(f'Distribution of {col}')
        plt.xticks(rotation=90)
        
        # Add count labels on bars
        for i, count in enumerate(value_counts.values):
            ax.text(i, count + 5, str(count), ha='center')
            
        # If target column is provided, show relationship
        if target_col and target_col in df.columns:
            plt.subplot(1, 2, 2)
            
            # Skip if there are too many categories
            if df[col].nunique() <= 30:
                # Box plot for target vs category
                sns.boxplot(x=col, y=target_col, data=df)
                plt.title(f'Relationship between {col} and {target_col}')
                plt.xticks(rotation=90)
                plt.tight_layout()
            else:
                plt.text(0.5, 0.5, f"Too many categories ({df[col].nunique()}) to display", 
                         ha='center', va='center', fontsize=14)
                
        plt.tight_layout()
        plt.show()

def visualize_categorical_relationships(df, categorical_cols=None, max_features=10):
    """
    Create a heatmap showing relationships between categorical features
    """
    if categorical_cols is None:
        categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    
    # Limit number of features for readability
    if len(categorical_cols) > max_features:
        # Select features with highest correlation to others
        categorical_cols = categorical_cols[:max_features]
    
    # Create a correlation matrix using Cramer's V
    n_cols = len(categorical_cols)
    cramers_v_matrix = np.zeros((n_cols, n_cols))
    
    # Function to calculate Cramer's V statistic
    def cramers_v(x, y):
        confusion_matrix = pd.crosstab(x, y)
        chi2 = 100  # Placeholder - actual calculation would use scipy.stats.chi2_contingency
        n = confusion_matrix.sum().sum()
        phi2 = chi2/n
        r, k = confusion_matrix.shape
        phi2corr = max(0, phi2-((k-1)*(r-1))/(n-1))
        rcorr = r-((r-1)**2)/(n-1)
        kcorr = k-((k-1)**2)/(n-1)
        return np.sqrt(phi2corr/min((kcorr-1), (rcorr-1)))
    
    # Fill the matrix
    for i, col1 in enumerate(categorical_cols):
        for j, col2 in enumerate(categorical_cols):
            if i == j:
                cramers_v_matrix[i, j] = 1.0
            else:
                # This is a placeholder - for real results use cramers_v(df[col1], df[col2])
                # We're using random values for demonstration purposes
                cramers_v_matrix[i, j] = np.random.uniform(0, 0.8)
    
    # Create the heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(cramers_v_matrix, annot=True, fmt=".2f", cmap="YlGnBu",
                xticklabels=categorical_cols, yticklabels=categorical_cols)
    plt.title('Relationship Between Categorical Features (Cramer\'s V)')
    plt.tight_layout()
    plt.show()
    

### full preprocessing_pipeline creation

In [10]:

# Custom transformer for applying WoE encoding
class WoECategoricalEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, n_bins=2, strategy='quantile'):
        self.n_bins = n_bins
        self.strategy = strategy
        self.binner = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy=strategy)
        self.woe_encoder = None
        
    def fit(self, X, y=None):
        if y is None:
            raise ValueError("WoECategoricalEncoder requires target values for fitting")
        
        # Bin the target
        y_binned = self.binner.fit_transform(y.values.reshape(-1, 1)).ravel()
        
        # Get categorical columns
        cat_cols = X.select_dtypes(include=['object']).columns.tolist()
        
        # Apply WoE encoding
        self.woe_encoder = WOEEncoder(cols=cat_cols)
        self.woe_encoder.fit(X[cat_cols], y_binned)
        
        return self
    
    def transform(self, X, y=None):
        X_transformed = X.copy()
        
        # Get categorical columns
        cat_cols = [col for col in self.woe_encoder.cols if col in X.columns]
        if cat_cols:
            X_transformed[cat_cols] = self.woe_encoder.transform(X[cat_cols])
            
        return X_transformed

## Correlation filtering

In [12]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import pandas as pd

# Custom transformer for correlation filtering
class CorrelationFilter(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.8, target_col=None):
        self.threshold = threshold
        self.target_col = target_col
        self.features_to_drop = []

    def fit(self, X, y=None):
        print("\n[CorrelationFilter] Fitting...")
        if self.target_col is not None and y is not None:
            print(f"Using target column '{self.target_col}' for guided correlation filtering.")
            data = X.copy()
            data[self.target_col] = y

            # Correlation with target
            target_corr = data.corr()[self.target_col].drop(self.target_col).abs()
            print("Correlation with target:")
            print(target_corr.sort_values(ascending=False))

            # Correlation between features
            corr_matrix = X.corr().abs()

            # Find highly correlated feature pairs
            high_corr_pairs = []
            for i in range(len(corr_matrix.columns)):
                for j in range(i + 1, len(corr_matrix.columns)):
                    if corr_matrix.iloc[i, j] > self.threshold:
                        feat1 = corr_matrix.columns[i]
                        feat2 = corr_matrix.columns[j]
                        high_corr_pairs.append((feat1, feat2))

            print(f"Highly correlated feature pairs (>{self.threshold}): {high_corr_pairs}")

            # Drop the one with lower correlation to target
            features_to_drop = []
            for feat1, feat2 in high_corr_pairs:
                drop_feat = feat1 if target_corr.get(feat1, 0) <= target_corr.get(feat2, 0) else feat2
                print(f"Between '{feat1}' and '{feat2}', dropping '{drop_feat}' (lower target corr)")
                features_to_drop.append(drop_feat)

            self.features_to_drop = list(set(features_to_drop))
            print("Final list of features to drop:", self.features_to_drop)

        else:
            print("No target column provided — applying unsupervised correlation filtering.")
            corr_matrix = X.corr().abs()
            mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

            high_corr_features = []
            for i in range(len(corr_matrix.columns)):
                for j in range(i + 1, len(corr_matrix.columns)):
                    if corr_matrix.iloc[i, j] > self.threshold:
                        feat1 = corr_matrix.columns[i]
                        feat2 = corr_matrix.columns[j]
                        print(f"High correlation between '{feat1}' and '{feat2}': {corr_matrix.iloc[i, j]}")
                        high_corr_features.append(feat2)

            self.features_to_drop = list(set(high_corr_features))
            print("Final list of features to drop:", self.features_to_drop)

        return self

    def transform(self, X, y=None):
        print("\n[CorrelationFilter] Transforming...")
        print(f"Dropping features: {self.features_to_drop}")
        return X.drop(columns=self.features_to_drop, errors='ignore')

    def get_feature_names_out(self, input_features=None):
        if input_features is None:
            raise ValueError("Input features not provided")
        return [f for f in input_features if f not in self.features_to_drop]


# Final Pipeline

In [54]:
# Now let's create the full pipeline
def create_house_price_pipeline(threshold=0.8):
    preprocessing_pipeline = Pipeline([
        ('data_cleaner', DataCleaner(target_column='SalePrice', null_threshold=0.90,
    variance_threshold=0.95, drop=True)),
        ('null_handler', NullHandler(numeric_strategy='median', categorical_strategy='mode', null_threshold=0.90)),
        ('one_hot_encoder', SelectiveOneHotEncoder(num_unique=3)),
        ('woe_encoder', WoECategoricalEncoder(n_bins=2, strategy='quantile')),
        # ('multi_woe_encoder', MultiClassWoEEncoder()),
        ('correlation_filter', CorrelationFilter(threshold=threshold, target_col='SalePrice'))
        
    ])
    
    return preprocessing_pipeline

In [25]:
from sklearn.preprocessing import OrdinalEncoder

def create_tree_model_pipeline(threshold=0.8):
    preprocessing_pipeline = Pipeline([
        ('data_cleaner', DataCleaner(
            target_column='SalePrice', 
            null_threshold=0.99,
            variance_threshold=0.95,
            drop=True
        )),
        ('null_handler', NullHandler(
            numeric_strategy='median', 
            categorical_strategy='mode', 
            null_threshold=0.99
        )),
        ('ordinal_encoder', OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)) # Custom encoder or use sklearn's OrdinalEncoder
        # ('correlation_filter', CorrelationFilter(
        #     threshold=threshold, 
        #     target_col='SalePrice'
        # ))
    ])
    
    return preprocessing_pipeline


In [64]:
import mlflow
from mlflow.sklearn import log_model, load_model

def process_with_pipeline(X_train, X_test, y_train, threshold=0.8):
    # Start MLflow run
    with mlflow.start_run(run_name="house_price_preprocessing") as run:
        # Create and fit pipeline
        pipeline = create_house_price_pipeline(threshold=threshold)
        X_train_processed = pipeline.fit_transform(X_train, y_train)
        X_test_processed = pipeline.transform(X_test)
        
        # Log parameters
        mlflow.log_param("correlation_threshold", threshold)
        mlflow.log_param("initial_feature_count", X_train.shape[1])
        mlflow.log_param("processed_feature_count", X_train_processed.shape[1])
        mlflow.log_param("dropped_feature_count", X_train.shape[1] - X_train_processed.shape[1])
        
        # Log the list of dropped features
        correlation_filter = pipeline.named_steps['correlation_filter']
        mlflow.log_param("dropped_features", ", ".join(correlation_filter.features_to_drop))
        
        # Save column names for reference (useful for inference)
        import pandas as pd
        if isinstance(X_train_processed, pd.DataFrame):
            feature_names = X_train_processed.columns.tolist()
        else:
            feature_names = [f"feature_{i}" for i in range(X_train_processed.shape[1])]
        
        pd.DataFrame({"feature_names": feature_names}).to_csv("feature_names.csv", index=False)
        mlflow.log_artifact("feature_names.csv")
        
        # Log the preprocessing pipeline
        # log_model(pipeline, "preprocessing_pipeline")
        
        # Get the run ID for later reference
        run_id = run.info.run_id

        import joblib
        joblib.dump(pipeline, "preprocessing_pipeline.joblib")
        mlflow.log_artifact("preprocessing_pipeline.joblib")
        
    return X_train_processed, X_test_processed, pipeline

In [50]:
import mlflow
import joblib
import os

mlflow.set_experiment("house_price_preprocessing_tree")

def process_with_tree_pipeline(X_train, X_test, y_train, threshold=0.8, pipeline_name="tree_preprocessing_pipeline.pkl"):
    # Start MLflow run
    with mlflow.start_run(run_name="tree_model_preprocessing"):
        # Create and fit pipeline
        pipeline = create_tree_model_pipeline(threshold=threshold)
        X_train_processed = pipeline.fit_transform(X_train, y_train)
        X_test_processed = pipeline.transform(X_test)
        
        # Log preprocessing stats
        mlflow.log_param("correlation_threshold", threshold)
        mlflow.log_param("initial_feature_count", X_train.shape[1])
        mlflow.log_param("processed_feature_count", X_train_processed.shape[1])
        mlflow.log_param("dropped_feature_count", X_train.shape[1] - X_train_processed.shape[1])
        
        # Log dropped features
        correlation_filter = pipeline.named_steps.get('correlation_filter')
        if correlation_filter and hasattr(correlation_filter, 'features_to_drop'):
            mlflow.log_param("dropped_features", ", ".join(correlation_filter.features_to_drop))

        # Save and log pipeline as artifact
        joblib.dump(pipeline, pipeline_name)
        mlflow.log_artifact(pipeline_name)
        
        return X_train_processed, X_test_processed, pipeline


In [17]:
mlflow.end_run()

# Training

## Train/Test Split

In [66]:
from sklearn.model_selection import train_test_split

X = df.copy()
y = X.pop('SalePrice')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [67]:
## Drop id
X_train = X_train.drop(columns='Id')
X_test = X_test.drop(columns='Id')

## Process with pipeline

In [68]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from category_encoders.woe import WOEEncoder


X_train_processed, X_test_processed, pipeline = process_with_pipeline(X_train, X_test, y_train, threshold=0.8)


Fitting DataCleaner...
Columns with null ratio > 0.9: ['Alley', 'PoolQC', 'MiscFeature']
Columns with same value ratio > 0.95: ['Street', 'Utilities', 'Condition2', 'RoofMatl', 'Heating', 'LowQualFinSF', 'KitchenAbvGr', 'GarageQual', 'GarageCond', '3SsnPorch', 'PoolArea', 'MiscVal']
Transforming dataset with DataCleaner...
Dropping columns: ['KitchenAbvGr', 'MiscVal', 'PoolArea', 'GarageCond', 'Alley', 'LowQualFinSF', 'PoolQC', 'Street', 'Utilities', 'GarageQual', '3SsnPorch', 'Heating', 'Condition2', 'RoofMatl', 'MiscFeature']
Shape after cleaning: (1168, 64)
Fitting NullHandler...
Columns with null ratio > 0.9: []
Numeric columns to fill: ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea',

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(



[CorrelationFilter] Fitting...
Using target column 'SalePrice' for guided correlation filtering.
Correlation with target:
OverallQual          0.785555
GrLivArea            0.695652
Neighborhood         0.681788
KitchenQual          0.645595
ExterQual            0.641619
                       ...   
BsmtHalfBath         0.048346
MasVnrType_BrkCmn    0.043160
MoSold               0.041890
YrSold               0.009099
BsmtFinSF2           0.005731
Name: SalePrice, Length: 73, dtype: float64
Highly correlated feature pairs (>0.8): [('Exterior1st', 'Exterior2nd'), ('TotalBsmtSF', '1stFlrSF'), ('GrLivArea', 'TotRmsAbvGrd'), ('GarageCars', 'GarageArea'), ('SaleType', 'SaleCondition'), ('LandSlope_Gtl', 'LandSlope_Mod'), ('MasVnrType_BrkFace', 'MasVnrType_Stone'), ('CentralAir_N', 'CentralAir_Y'), ('PavedDrive_N', 'PavedDrive_Y')]
Between 'Exterior1st' and 'Exterior2nd', dropping 'Exterior2nd' (lower target corr)
Between 'TotalBsmtSF' and '1stFlrSF', dropping '1stFlrSF' (lower target corr)

In [ ]:
X_train_processed.shape

In [28]:
# For tree models

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from category_encoders.woe import WOEEncoder
from sklearn.preprocessing import OrdinalEncoder



X_train_processed, X_test_processed, tree_pipeline = process_with_tree_pipeline(X_train, X_test, y_train)


Fitting DataCleaner...
Columns with null ratio > 0.99: ['PoolQC']
Columns with same value ratio > 0.95: ['Street', 'Utilities', 'Condition2', 'RoofMatl', 'Heating', 'LowQualFinSF', 'KitchenAbvGr', 'GarageQual', 'GarageCond', '3SsnPorch', 'PoolArea', 'MiscVal']
Transforming dataset with DataCleaner...
Dropping columns: ['KitchenAbvGr', 'MiscVal', 'PoolArea', 'GarageCond', 'LowQualFinSF', 'PoolQC', 'Street', 'Utilities', 'GarageQual', '3SsnPorch', 'Heating', 'Condition2', 'RoofMatl']
Shape after cleaning: (1168, 66)
Fitting NullHandler...
Columns with null ratio > 0.99: []
Numeric columns to fill: ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch',

# linear models

## Linear Regression

In [ ]:
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_squared_log_error
import seaborn as sns
import matplotlib.pyplot as plt

# Set MLflow experiment
experiment_name = "House Price Regression"
mlflow.set_experiment(experiment_name)

# Initialize model
model = LinearRegression()

# Start MLflow run
with mlflow.start_run() as run:
    # Set a custom name for the run
    mlflow.set_tag("mlflow.runName", "Linear_Regression_v2")

    # Log model parameters (none for basic LinearRegression, but for demo let's log default ones)
    mlflow.log_param("fit_intercept", model.fit_intercept)
    mlflow.log_param("model_type", "LinearRegression")

    # Train the model
    model.fit(X_train_processed, y_train)
    y_pred = model.predict(X_test_processed)

    y_pred_clipped = np.maximum(y_pred, 1)
    y_test_clipped = np.maximum(y_test, 1)

    # Calculate metrics
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    rmsle = np.sqrt(mean_squared_log_error(y_test_clipped, y_pred_clipped))

    # Log metrics
    mlflow.log_metric("MAE", mae)
    mlflow.log_metric("MSE", mse)
    mlflow.log_metric("RMSE", rmse)
    mlflow.log_metric("R2", r2)
    mlflow.log_metric("RMSLE", rmsle)

    # Log the model
    mlflow.sklearn.log_model(model, artifact_path="model", registered_model_name="HousePricesLinearModel")

    # Plot and save prediction vs actual plot
    plt.figure(figsize=(6, 6))
    sns.scatterplot(x=y_test, y=y_pred)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.xlabel("Actual Prices")
    plt.ylabel("Predicted Prices")
    plt.title("Predicted vs Actual Prices")
    plt.tight_layout()
    plt.savefig("scatter_plot.png")

    # Log the plot image
    mlflow.log_artifact("scatter_plot.png")
    mlflow.end_run()

In [ ]:
import numpy as np
import mlflow
import mlflow.sklearn
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_squared_log_error

# Set MLflow experiment
experiment_name = "House Price Regression"
mlflow.set_experiment(experiment_name)

# Number of folds
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

fold = 1
for train_index, test_index in kf.split(X_train_processed):
    with mlflow.start_run(run_name=f"LinearRegression_Fold_{fold}"):
        # Split the data
        X_train, X_test = X_train_processed.iloc[train_index], X_train_processed.iloc[test_index]  # .iloc for DataFrame
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]  # .iloc for Series/DataFrame

        # Initialize model
        model = LinearRegression()

        # Log parameters
        mlflow.log_param("fit_intercept", model.fit_intercept)
        mlflow.log_param("model_type", "LinearRegression")
        mlflow.log_param("fold", fold)

        # Train the model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Clip negative predictions for RMSLE
        y_pred_clipped = np.maximum(y_pred, 1)
        y_test_clipped = np.maximum(y_test, 1)

        # Metrics
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_pred)
        rmsle = rmsle(y_test_clipped, y_pred_clipped)

        # Log metrics
        mlflow.log_metric("MAE", mae)
        mlflow.log_metric("MSE", mse)
        mlflow.log_metric("RMSE", rmse)
        mlflow.log_metric("R2", r2)
        mlflow.log_metric("RMSLE", rmsle)

        # Log model
        mlflow.sklearn.log_model(model, artifact_path=f"model_fold_{fold}")

        # Plot predictions
        plt.figure(figsize=(6, 6))
        sns.scatterplot(x=y_test, y=y_pred)
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
        plt.xlabel("Actual Prices")
        plt.ylabel("Predicted Prices")
        plt.title(f"Predicted vs Actual Prices (Fold {fold})")
        plt.tight_layout()

        plot_name = f"scatter_plot_fold_{fold}.png"
        plt.savefig(plot_name)
        mlflow.log_artifact(plot_name)

        fold += 1


## Linear Regression with GridSearch and RFE

In [ ]:
X_train_processed.head()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import mlflow
import mlflow.sklearn
import numpy as np
import pandas as pd

experiment_name = "House Price Regression"
run_name = "Linear_Regression_RFE_v1"
mlflow.set_experiment(experiment_name)

base_model = LinearRegression()

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('feature_selector', RFE(estimator=base_model)),
    ('regressor', base_model)
])

param_grid = {
    'feature_selector__n_features_to_select': [20, 25, 40, 60],
}


grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_root_mean_squared_error')

# MLflow tracking
with mlflow.start_run(run_name=run_name) as run:
    # Fit the model
    grid_search.fit(X_train_processed, y_train)
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_
    
    # Get predictions
    y_pred = best_model.predict(X_test_processed)
    
    # Metrics
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    y_test_clipped = np.maximum(0, y_test)
    y_pred_clipped = np.maximum(0, y_pred)
    log_rmse = np.sqrt(mean_squared_error(np.log1p(y_test_clipped), np.log1p(y_pred_clipped)))

    
    # Log model type
    mlflow.log_param("model_type", "LinearRegression")
    
    # Log best parameters from GridSearch
    for param_name, param_value in best_params.items():
        mlflow.log_param(param_name, param_value)
    
    # NEW: Log the selected features
    feature_selector = best_model.named_steps['feature_selector']
    selected_features_mask = feature_selector.support_
    feature_names = X_train_processed.columns.tolist()
    selected_features = [feature for feature, selected in zip(feature_names, selected_features_mask) if selected]
    
    # Log selected features as a parameter (as string)
    mlflow.log_param("selected_features", str(selected_features))
    
    # Save and log the selected features as a CSV
    selected_features_df = pd.DataFrame({
        'feature_name': feature_names,
        'selected': selected_features_mask
    })
    selected_features_path = "selected_features.csv"
    selected_features_df.to_csv(selected_features_path, index=False)
    mlflow.log_artifact(selected_features_path)
    
    # Log metrics
    mlflow.log_metric("MAE", mae)
    mlflow.log_metric("MSE", mse)
    mlflow.log_metric("RMSE", rmse)
    mlflow.log_metric("R2", r2)
    mlflow.log_metric("Log_RMSE", log_rmse)

    
    # NEW: Log optimized RMSE (with validation scores)
    cv_rmse_scores = np.sqrt(-grid_search.cv_results_['mean_test_score'])
    best_cv_rmse = cv_rmse_scores[grid_search.best_index_]
    mlflow.log_metric("best_cv_rmse", best_cv_rmse)
    mlflow.log_metric("rmse_std", np.std(cv_rmse_scores))
    
    # Log the final model
    mlflow.sklearn.log_model(best_model, "model", registered_model_name="HousePricesLinearModel_RFE")

    from sklearn.inspection import permutation_importance
    
    result = permutation_importance(best_model, X_test_processed, y_test, n_repeats=10, random_state=42)
    
    perm_sorted_idx = result.importances_mean.argsort()[::-1]
    features = np.array(X_train_processed.columns)[perm_sorted_idx]
    importances = result.importances_mean[perm_sorted_idx]
    
    # Plot and log
    plt.figure(figsize=(10, 6))
    plt.barh(features, importances)
    plt.xlabel("Permutation Importance")
    plt.title("Permutation Feature Importance")
    plt.tight_layout()
    plt.savefig("permutation_importance.png")
    mlflow.log_artifact("permutation_importance.png")


    import shap

    explainer = shap.Explainer(best_model.named_steps['regressor'], X_train_processed[selected_features])
    shap_values = explainer(X_test_processed[selected_features])
    
    # Summary plot
    shap.summary_plot(shap_values, X_test_processed[selected_features], show=False)
    plt.tight_layout()
    plt.savefig("shap_summary.png")
    mlflow.log_artifact("shap_summary.png")

    
    
    # Log feature importances if available
    try:
        if hasattr(best_model.named_steps['regressor'], 'coef_'):
            # For linear models, we can get coefficients
            coef = best_model.named_steps['regressor'].coef_
            # Only get coefficients for selected features
            selected_feature_coefs = [(feature, coef) for feature, selected, coef in 
                                     zip(feature_names, selected_features_mask, coef) if selected]
            
            # Create and log feature importance plot
            import matplotlib.pyplot as plt
            plt.figure(figsize=(10, 6))
            features, importance = zip(*selected_feature_coefs)
            plt.barh(features, importance)
            plt.xlabel('Coefficient Value')
            plt.title('Feature Importance')
            plt.tight_layout()
            plt.savefig('feature_importance.png')
            mlflow.log_artifact('feature_importance.png')
    except Exception as e:
        # Log any errors that occur when extracting feature importance
        mlflow.log_param("feature_importance_error", str(e))

## Ridge Regression

In [ ]:
import numpy as np
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.linear_model import Ridge
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# Set up MLflow experiment
experiment_name = "House Price Regression"
run_name = "Ridge_RFE_v3"
experiment_description = "Grid search for Ridge regression with RFE using different alpha values, optimizing for RMSE log"

# Check if experiment exists, otherwise create it
try:
    experiment_id = mlflow.create_experiment(
        experiment_name, 
        tags={"description": experiment_description}
    )
except mlflow.exceptions.MlflowException:
    experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id

# Set the active experiment
mlflow.set_experiment(experiment_name)

# Define a custom scorer that calculates RMSE on log-transformed values
def rmse_log_scorer(estimator, X, y):
    y_pred = estimator.predict(X)
    # Handle potential negative predictions
    y_pred = np.maximum(y_pred, 1e-10)  # Set a small positive minimum value
    log_y = np.log(y)
    log_y_pred = np.log(y_pred)
    mse_log = mean_squared_error(log_y, log_y_pred)
    rmse_log = np.sqrt(mse_log)
    return -rmse_log  # Negative because GridSearchCV maximizes score

# Define the parameter grid
param_grid = {
    'ridge__alpha': [50.0, 60.0, 80.0, 100.0],
    'rfe__n_features_to_select': [54, 55, 56]  
}

# Create pipeline with RFE and Ridge
ridge = Ridge()
rfe = RFE(estimator=Ridge(), step=1)
pipeline = Pipeline([
    ('rfe', rfe),
    ('ridge', ridge)
])

# Create the grid search
grid_search = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    scoring=rmse_log_scorer,
    cv=5,
    verbose=1,
    n_jobs=-1  # Use all available cores
)

# Assuming X_train_processed and y_train are your training data
# Fit the grid search
print("Starting grid search...")
grid_search.fit(X_train_processed, y_train)
print("Grid search completed!")

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = -grid_search.best_score_  # Convert back to positive RMSE
print(f"Best parameters: {best_params}")
print(f"Best RMSE log score: {best_score:.4f}")

# Get the best model
best_model = grid_search.best_estimator_

# Evaluate the best model on test data
y_pred = best_model.predict(X_test_processed)

# Calculate metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Calculate RMSE log
y_pred = np.maximum(y_pred, 1e-10)  # Ensure positive values
log_y_test = np.log(y_test)
log_y_pred = np.log(y_pred)
mse_log = mean_squared_error(log_y_test, log_y_pred)
rmse_log = np.sqrt(mse_log)

print(f"Test MAE:     {mae:.4f}")
print(f"Test MSE:     {mse:.4f}")
print(f"Test RMSE:    {rmse:.4f}")
print(f"Test R²:      {r2:.4f}")
print(f"Test RMSE_log: {rmse_log:.4f}")

# Log the best model and results to MLflow
with mlflow.start_run(run_name=run_name):
    # Log all parameters
    mlflow.log_param("alpha", best_params['ridge__alpha'])
    mlflow.log_param("n_features_selected", best_params['rfe__n_features_to_select'])
    mlflow.log_param("total_features", X_train_processed.shape[1])
    
    # Log metrics
    mlflow.log_metrics({
        "mae": mae,
        "mse": mse,
        "rmse": rmse,
        "r2": r2,
        "rmse_log": rmse_log,
        "cv_rmse_log": best_score
    })
    
    # Log all grid search results as artifacts
    cv_results = pd.DataFrame(grid_search.cv_results_)
    cv_results.to_csv("grid_search_results.csv", index=False)
    mlflow.log_artifact("grid_search_results.csv")
    
    # Log the best model
    mlflow.sklearn.log_model(best_model, "ridge_rfe_model")
    
    # Log which features were selected
    if hasattr(best_model.named_steps['rfe'], 'support_'):
        selected_features = np.where(best_model.named_steps['rfe'].support_)[0]
        feature_ranking = best_model.named_steps['rfe'].ranking_
        
        # If you have feature names available, use them
        # Assuming feature_names is a list of feature names
        try:
            feature_names = X_train_processed.columns.tolist()
            selected_feature_names = [feature_names[i] for i in selected_features]
            with open("selected_features.txt", "w") as f:
                f.write("Selected features:\n")
                for name in selected_feature_names:
                    f.write(f"- {name}\n")
                
                f.write("\nFeature rankings:\n")
                for i, name in enumerate(feature_names):
                    f.write(f"- {name}: {feature_ranking[i]}\n")
            
            mlflow.log_artifact("selected_features.txt")
        except:
            # If feature names are not available
            np.savetxt("selected_features.txt", selected_features)
            np.savetxt("feature_ranking.txt", feature_ranking)
            mlflow.log_artifact("selected_features.txt")
            mlflow.log_artifact("feature_ranking.txt")
    
    # Log model type and description
    mlflow.set_tags({
        "model_type": "Ridge Regression with RFE",
        "description": "Grid search over alpha values and number of features using RFE",
        "optimization_metric": "RMSE on log-transformed predictions"
    })
    
    run_id = mlflow.active_run().info.run_id
    print(f"Run ID: {run_id}")
    print(f"Experiment ID: {experiment_id}")
    print("Model and results successfully logged to MLflow!")

## Lasso Regression (L1 Regularization)

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import mlflow
import mlflow.sklearn
import numpy as np
import pandas as pd

experiment_name = "House Price Regression"
run_name = "Lasso_Regression_v4"
mlflow.set_experiment(experiment_name)

# Base model - Lasso regression
base_model = Lasso()

# Pipeline
# Modified pipeline with adjusted parameters
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('feature_selector', RFE(estimator=Lasso(max_iter=10000, tol=0.01))),
    ('regressor', Lasso(max_iter=10000, tol=0.01))  # Increased max_iter and relaxed tolerance
])

# Modified parameter grid with more reasonable alpha values
param_grid = {
    'feature_selector__n_features_to_select': [18, 20, 22],
    'regressor__alpha': [ 1500.0, 2000.0, 2500.0],  # Lower alpha values to try
}

# GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_root_mean_squared_error')

# MLflow tracking
with mlflow.start_run(run_name=run_name) as run:
    # Fit the model
    grid_search.fit(X_train_processed, y_train)
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_
    
    # Get predictions
    y_pred = best_model.predict(X_test_processed)
    
    # Metrics
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    y_test_clipped = np.maximum(0, y_test)
    y_pred_clipped = np.maximum(0, y_pred)
    log_rmse = np.sqrt(mean_squared_error(np.log1p(y_test_clipped), np.log1p(y_pred_clipped)))

    
    # Log model type
    mlflow.log_param("model_type", "LassoRegression")
    
    # Log best parameters from GridSearch
    for param_name, param_value in best_params.items():
        mlflow.log_param(param_name, param_value)
    
    # Log the selected features
    feature_selector = best_model.named_steps['feature_selector']
    selected_features_mask = feature_selector.support_
    feature_names = X_train_processed.columns.tolist()
    selected_features = [feature for feature, selected in zip(feature_names, selected_features_mask) if selected]
    
    # Log selected features as a parameter (as string)
    mlflow.log_param("selected_features", str(selected_features))
    
    # Save and log the selected features as a CSV
    selected_features_df = pd.DataFrame({
        'feature_name': feature_names,
        'selected': selected_features_mask
    })
    selected_features_path = "selected_features.csv"
    selected_features_df.to_csv(selected_features_path, index=False)
    mlflow.log_artifact(selected_features_path)
    
    # Log metrics
    mlflow.log_metric("MAE", mae)
    mlflow.log_metric("MSE", mse)
    mlflow.log_metric("RMSE", rmse)
    mlflow.log_metric("R2", r2)
    mlflow.log_metric("Log_RMSE", log_rmse)

    # Log optimized RMSE (with validation scores)
    cv_rmse_scores = np.sqrt(-grid_search.cv_results_['mean_test_score'])
    best_cv_rmse = cv_rmse_scores[grid_search.best_index_]
    mlflow.log_metric("best_cv_rmse", best_cv_rmse)
    mlflow.log_metric("rmse_std", np.std(cv_rmse_scores))
    
    # Log the final model
    mlflow.sklearn.log_model(best_model, "model", registered_model_name="HousePricesLassoModel_RFE")



# Tree Models

In [38]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import mlflow
import mlflow.sklearn  # Optional
import numpy as np


In [39]:
def evaluate_model(model, model_name, X_train, y_train, X_test, y_test, log_mlflow=False):
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = mean_squared_error(y_test, preds, squared=False)
    r2 = r2_score(y_test, preds)

    print(f"{model_name} RMSE: {rmse:.4f}, R2: {r2:.4f}")

    if log_mlflow:
        with mlflow.start_run(run_name=model_name):
            mlflow.log_param("model", model_name)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            
            mlflow.sklearn.log_model(model, model_name)

    return model, rmse, r2


In [40]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

models = {
    "RandomForest": RandomForestRegressor(random_state=42),
    "GradientBoosting": GradientBoostingRegressor(random_state=42),
    "Bagging": BaggingRegressor(random_state=42),
    "DecisionTree": DecisionTreeRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42, objective='reg:squarederror')
}

param_grids = {
    "RandomForest": {
        "n_estimators": [150, 200, 250, 300],
        "max_depth": [15, 20, 25, 30]
    },
    "GradientBoosting": {
        "n_estimators": [150, 200, 250],
        "learning_rate": [0.05, 0.1, 1, 10.0],
        "max_depth": [2, 3, 4]
    },
    "Bagging": {
        "n_estimators": [10, 50],
        "max_samples": [0.5, 1.0]
    },
    "DecisionTree": {
        "max_depth": [None, 10, 20],
        "min_samples_split": [2, 10]
    },
    "XGBoost": {
        "n_estimators": [150, 200, 250],
        "learning_rate": [0.05, 0.1, 1],
        "max_depth": [2, 3, 4]
    }
}


In [41]:
from sklearn.metrics import make_scorer, mean_squared_log_error
import numpy as np

def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, np.maximum(0, y_pred)))

rmsle_scorer = make_scorer(rmsle, greater_is_better=False)


In [43]:
from sklearn.model_selection import GridSearchCV
import mlflow
import pandas as pd

results_df = []

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    grid = GridSearchCV(model, param_grids[name], scoring=rmsle_scorer, cv=5, n_jobs=-1)
    grid.fit(X_train_processed, y_train)

    best_model = grid.best_estimator_
    preds = best_model.predict(X_test_processed)
    rmsle_score = rmsle(y_test, preds)

    residuals = y_test - preds

    # MLflow tracking
    with mlflow.start_run(run_name=name) as run:
        mlflow.log_params(grid.best_params_)
        mlflow.log_metric("RMSLE", rmsle_score)

        # Feature importance
        if hasattr(best_model, 'feature_importances_'):
            importances = best_model.feature_importances_

            # Ensure the column names match the transformed features after encoding
            # If using OneHotEncoder, get the feature names correctly
            if hasattr(X_train_processed, 'columns'):  # If X_train_processed is a DataFrame
                feature_names = X_train_processed.columns
            else:  # If it's a NumPy array, use range indices
                feature_names = [f"Feature_{i}" for i in range(len(importances))]

            # Create the feature importance DataFrame
            feat_imp_df = pd.DataFrame({
                'Feature': feature_names,
                'Importance': importances
            }).sort_values(by='Importance', ascending=False)

            # Log feature importance
            feat_imp_csv = f"{name}_feature_importance.csv"
            feat_imp_df.to_csv(feat_imp_csv, index=False)
            mlflow.log_artifact(feat_imp_csv)

        # Log predictions
        pred_df = pd.DataFrame({
            'Actual': y_test,
            'Predicted': preds,
            'Residual': residuals
        })
        pred_csv = f"{name}_predictions.csv"
        pred_df.to_csv(pred_csv, index=False)
        mlflow.log_artifact(pred_csv)

        results_df.append({
            'Model': name,
            'BestParams': grid.best_params_,
            'RMSLE': rmsle_score
        })

        model_name = f"HousePrice_{name}"
        mlflow.sklearn.log_model(best_model, artifact_path="model", registered_model_name=model_name)

        results_df.append({
            'Model': name,
            'BestParams': grid.best_params_,
            'RMSLE': rmsle_score,
            'MLflowRunID': run.info.run_id
        })



Training RandomForest...


2025/04/10 19:26:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'HousePrice_RandomForest' already exists. Creating a new version of this model...
2025/04/10 19:26:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: HousePrice_RandomForest, version 2
Created version '2' of model 'HousePrice_RandomForest'.


🏃 View run RandomForest at: https://dagshub.com/ekvirika/HousePrices.mlflow/#/experiments/5/runs/c3e07e871a5b408ebbf79c51a4a583f5
🧪 View experiment at: https://dagshub.com/ekvirika/HousePrices.mlflow/#/experiments/5

Training GradientBoosting...


2025/04/10 19:27:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'HousePrice_GradientBoosting'.
2025/04/10 19:27:54 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: HousePrice_GradientBoosting, version 1
Created version '1' of model 'HousePrice_GradientBoosting'.


🏃 View run GradientBoosting at: https://dagshub.com/ekvirika/HousePrices.mlflow/#/experiments/5/runs/80a8224d627447ad9535932195a1ec4c
🧪 View experiment at: https://dagshub.com/ekvirika/HousePrices.mlflow/#/experiments/5

Training Bagging...


...

🏃 View run Bagging at: https://dagshub.com/ekvirika/HousePrices.mlflow/#/experiments/5/runs/6e2c588e2a9a495daf62797cbd63e3fb
🧪 View experiment at: https://dagshub.com/ekvirika/HousePrices.mlflow/#/experiments/5

Training DecisionTree...


2025/04/10 19:28:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'HousePrice_DecisionTree'.
2025/04/10 19:28:22 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: HousePrice_DecisionTree, version 1
Created version '1' of model 'HousePrice_DecisionTree'.


🏃 View run DecisionTree at: https://dagshub.com/ekvirika/HousePrices.mlflow/#/experiments/5/runs/d71f08e7b7ee42619009f1af5ad202d3
🧪 View experiment at: https://dagshub.com/ekvirika/HousePrices.mlflow/#/experiments/5

Training XGBoost...


2025/04/10 19:28:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'HousePrice_XGBoost'.
2025/04/10 19:28:47 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: HousePrice_XGBoost, version 1
Created version '1' of model 'HousePrice_XGBoost'.


🏃 View run XGBoost at: https://dagshub.com/ekvirika/HousePrices.mlflow/#/experiments/5/runs/2fda25842af5499fb0986be19a5dc6d1
🧪 View experiment at: https://dagshub.com/ekvirika/HousePrices.mlflow/#/experiments/5


# Submission

In [ ]:
complete_pipeline = Pipeline([
    ('pipeline', pipeline),
    ('model', best_model)
])

In [ ]:
import pandas as pd

test_data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')  # Replace with your actual file path

# If your test data needs ID columns preserved for submission
id_column = test_data['Id']  # Replace 'id' with your actual ID column name
test_features = test_data.drop(['Id'], axis=1)  # Adjust column names as needed
predictions = complete_pipeline.predict(test_features)

submission = pd.DataFrame({
    'id': id_column,
    'SalePrice': predictions
})

# Save to CSV
submission.to_csv('submission.csv', index=False)
print("Submission file created successfully!")